This notebook reproduces creation of toxic and normal corpus


# Requeriments

In [1]:
import pandas as pd
import emoji
import re
import string
import spacy
import es_core_news_sm
nlp = spacy.load("es_core_news_sm")
import preprocessor as p
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [2]:
!python -m spacy download es_core_news_sm

DEPRECATION: https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-2.3.1/es_core_news_sm-2.3.1.tar.gz#egg=es_core_news_sm==2.3.1 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 29.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [3]:
tqdm.pandas()  # Initialize tqdm for pandas

# Cargando datos

In [4]:
path_raw = './raw/'

In [5]:

ruta_archivo1 = path_raw + 'clean_comentarios_facebook.csv'
ruta_archivo2 = path_raw + 'labeled_corpus_6K.txt'
ruta_archivo3 = path_raw + 'sp_tweets.csv'
ruta_archivo4 = path_raw + 'train_es.tsv'
ruta_archivo5 = path_raw +'hascosva_2022_anonymized.tsv'

In [6]:
data1 = pd.read_csv(ruta_archivo1, sep=',', engine='python')
data2 = pd.read_csv(ruta_archivo2, sep=';\\|\\|;', engine='python', names=['id','twt', 'label'])
data3 = pd.read_csv(ruta_archivo3, sep=',', engine='python')
data4 = pd.read_csv(ruta_archivo4, sep='\t')
data5 = pd.read_csv(ruta_archivo5, sep='\t')

In [7]:
toxicData = []
normalData = []

normalData.extend(data1.loc[data1['Category'] == 0, 'Text'].tolist())
toxicData.extend(data1.loc[data1['Category'] == 1, 'Text'].tolist())

normalData.extend(data2.loc[data2['label'] == 0, 'twt'].tolist())
toxicData.extend(data2.loc[data2['label'] == 1, 'twt'].tolist())

toxicData.extend(data3['tweet'].tolist())

normalData.extend(data4.loc[data4['HS'] == 0, 'text'].tolist())
toxicData.extend(data4.loc[data4['HS'] == 1, 'text'].tolist())

normalData.extend(data5.loc[data5['label'] == 0, 'text'].tolist())
toxicData.extend(data5.loc[data5['label'] == 1, 'text'].tolist())


In [8]:
pdToxic = pd.DataFrame({'twt': toxicData, 'label': 1})
pdNormal = pd.DataFrame({'twt': normalData, 'label': 0})

In [9]:
df =  pd.concat([pdToxic, pdNormal], ignore_index=True)
df["label"] =  df["label"].astype(bool)
df["twt"] =  df["twt"].astype(str)

In [10]:
df["label"].value_counts()

True     110678
False     12181
Name: label, dtype: int64

# Pre procesamiento

In [11]:
df.head()

,twt,label
0,y pensar ganar respetar ver aborrencimiento de...,True
1,a a holgar ver hombre necesitar coser obvio ne...,True
2,drogar mocoso q empezar a hablar reportaje met...,True
3,mujer feminazi destruir año verdadero luchar f...,True
4,feminista feminoide,True


## Limpiando texto

In [61]:
def clean_text(text):
    # Clean with tweet preprocessor: remove URLs, Hashtags, Mentions, Emojis, Smileys
    text = p.clean(text) 

    # Convert text to lowercase, remove punctuation, characters with numbers, and newlines
    text = text.lower()
    text = re.sub(r'.,;*?¿', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'(\w)\1{2,}', r'\1\1', text)

    # Process the text with spaCy: remove stopwords and lemmatize
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    text = " ".join(tokens)

    # Remove extra whitespaces
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    # Remove words shorter than 2 letters
    words = text.split() 
    filtered_words = [word for word in words if len(word) > 2]
    text = ' '.join(filtered_words) 

    return text




In [62]:
df['twt_cleaned'] = df['twt'].progress_apply(clean_text)
df

100%|██████████| 122859/122859 [13:08<00:00, 155.76it/s]


,twt,label,twt_cleaned,twt_corr
0,y pensar ganar respetar ver aborrencimiento de...,True,pensar ganar respetar aborrencimiento demasiar...,Y pensar ganar respetar aborrecimiento demasia...
1,a a holgar ver hombre necesitar coser obvio ne...,True,holgar hombre necesitar coser obviar necesitar...,a holgar hombre necesitar coser obviar necesit...
2,drogar mocoso q empezar a hablar reportaje met...,True,drogar mocoso empezar hablar reportaje meter p...,Drogar mocoso que empezar a hablar reportaje m...
3,mujer feminazi destruir año verdadero luchar f...,True,mujer feminazi destruir luchar feminista,Mujer féminas destruir a luchar feminista
4,feminista feminoide,True,feminista feminoide,Feminista feminoide
...,...,...,...,...
122854,Un debate interesante ¿las mujeres occidentale...,False,debatir interesante mujer occidental decidir l...,Debatir interesante mujer occidental decidir l...
122855,@ofya @ogsn De todo esto y leyendo las respues...,False,leer respuesta quen madurez mensaje correcto p...,Y leer respuesta que madurez y mensaje correct...
122856,@oxqh @oajv la reina respeta la religión musul...,False,reinar respetar religin musulmán marrueco resp...,Reinar respetar religión musulmán marrueco y r...
122857,"""Las decisiones económicas son exclusivas del ...",False,decisión econmicas exclusivo organización prof...,Decisión económicas exclusivo y organización p...


In [63]:
import numpy as np
df.replace('', np.nan, inplace=True)
df = df.dropna()
df.reset_index(inplace=True)
df

,index,twt,label,twt_cleaned,twt_corr
0,0,y pensar ganar respetar ver aborrencimiento de...,True,pensar ganar respetar aborrencimiento demasiar...,Y pensar ganar respetar aborrecimiento demasia...
1,1,a a holgar ver hombre necesitar coser obvio ne...,True,holgar hombre necesitar coser obviar necesitar...,a holgar hombre necesitar coser obviar necesit...
2,2,drogar mocoso q empezar a hablar reportaje met...,True,drogar mocoso empezar hablar reportaje meter p...,Drogar mocoso que empezar a hablar reportaje m...
3,3,mujer feminazi destruir año verdadero luchar f...,True,mujer feminazi destruir luchar feminista,Mujer féminas destruir a luchar feminista
4,4,feminista feminoide,True,feminista feminoide,Feminista feminoide
...,...,...,...,...,...
121533,122854,Un debate interesante ¿las mujeres occidentale...,False,debatir interesante mujer occidental decidir l...,Debatir interesante mujer occidental decidir l...
121534,122855,@ofya @ogsn De todo esto y leyendo las respues...,False,leer respuesta quen madurez mensaje correcto p...,Y leer respuesta que madurez y mensaje correct...
121535,122856,@oxqh @oajv la reina respeta la religión musul...,False,reinar respetar religin musulmán marrueco resp...,Reinar respetar religión musulmán marrueco y r...
121536,122857,"""Las decisiones económicas son exclusivas del ...",False,decisión econmicas exclusivo organización prof...,Decisión económicas exclusivo y organización p...


In [64]:
with open('./processed/normalCorpusWSpellErr.txt', 'w') as normal, open('./processed/toxicCorpusWSpellErr.txt', 'w') as toxic:
    for twt in df[df['label']==0].twt_cleaned:
        normal.writelines(f'{twt}\n')
    for twt in df[df['label']==1].twt_cleaned:
        toxic.writelines(f'{twt}\n')

# Corrigiendo palabras

In [25]:
import language_tool_python
tool = language_tool_python.LanguageTool('es')

Unzipping /tmp/tmpc8dutqwj.zip to /home/gabriel/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /home/gabriel/.cache/language_tool_python.


In [30]:
def correct_text(text):
    return tool.correct(text)

In [31]:
df['twt_corr'] = df['twt_cleaned'].progress_apply(correct_text)
df

100%|██████████| 122859/122859 [54:10<00:00, 37.80it/s] 


,twt,label,twt_cleaned,twt_corr
0,y pensar ganar respetar ver aborrencimiento de...,True,y pensar ganar respetar aborrencimiento demasi...,Y pensar ganar respetar aborrecimiento demasia...
1,a a holgar ver hombre necesitar coser obvio ne...,True,a a holgar hombre necesitar coser obviar neces...,a holgar hombre necesitar coser obviar necesit...
2,drogar mocoso q empezar a hablar reportaje met...,True,drogar mocoso q empezar a hablar reportaje met...,Drogar mocoso que empezar a hablar reportaje m...
3,mujer feminazi destruir año verdadero luchar f...,True,mujer feminazi destruir ao luchar feminista,Mujer féminas destruir a luchar feminista
4,feminista feminoide,True,feminista feminoide,Feminista feminoide
...,...,...,...,...
122854,Un debate interesante ¿las mujeres occidentale...,False,debatir interesante mujer occidental decidir l...,Debatir interesante mujer occidental decidir l...
122855,@ofya @ogsn De todo esto y leyendo las respues...,False,y leer respuesta quen madurez y mensaje correc...,Y leer respuesta que madurez y mensaje correct...
122856,@oxqh @oajv la reina respeta la religión musul...,False,reinar respetar religin musulmán marrueco y re...,Reinar respetar religión musulmán marrueco y r...
122857,"""Las decisiones económicas son exclusivas del ...",False,decisión econmicas exclusivo y organización pr...,Decisión económicas exclusivo y organización p...


In [40]:
with open('./processed/normalCorpusSpellErr.txt', 'w') as normal, open('./processed/toxicCorpusSpellErr.txt', 'w') as toxic:
    for twt in df_clean[df_clean['label']==0].twt_corr:
        normal.writelines(f'{twt}\n')
    for twt in df_clean[df_clean['label']==1].twt_corr:
        toxic.writelines(f'{twt}\n')